In [0]:
import requests
from datetime import datetime

# Databricks notebook source
# Descarga GDELT GKG v2.1 (.gkg.csv.zip), descomprime y guarda como .csv en landing.
# Luego, tu Auto Loader (bronze_gdelt_autoloader.py) toma esos .csv y los carga a bronze.gdelt_gkg_raw.

BASE = "https://api.gdeltproject.org/api/v2/doc/doc"

def gdelt_gkg_url(company_alias: list[str], ts: str) -> str:
    # Construye la parte de la query con los alias
    query = "%20OR%20".join([f'"{alias}"' for alias in company_alias])
    query = f"({query})"
    return f"{BASE}?query={query}&ts={ts}&mode=tonechart&format=json"

downloaded, skipped, errors = 0, 0, []

for c in companies:
    try:
        # company_alias debe ser una lista de strings, por ejemplo: ["NVIDIA", "Nvidia"]
        url = gdelt_gkg_url(c.company_alias, "24hs")

        response = requests.get(url)
        response.raise_for_status()
        data = response.json()

        # Extrae la fecha actual (puedes ajustar si quieres la fecha de la request)
        date = datetime.now().strftime("%Y-%m-%d")

        # Construye las filas para cada bin
        rows = []
        for bin_info in data.get("tonechart", []):
            rows.append({
            "company_name": c.company_name,
            "date": date,
            "bin": bin_info["bin"],
            "count": bin_info["count"]
            })

        # Si hay filas, crea el DataFrame y guárdalo (puedes cambiar el nombre del archivo/tabla)
        if rows:
            df = spark.createDataFrame(rows)
            df.write.mode("append").saveAsTable("bronze.gdelt_tonechart")
        else:
            skipped += 1
        downloaded += 1
    except Exception as e:
        errors.append((c.company_name, str(e)))

print(f"Descargados: {downloaded} | Saltados: {skipped} | Errores: {len(errors)}")
if errors:
    display(spark.createDataFrame(errors, schema="ts string, error string"))